In [ ]:
import json

In [ ]:

with open('eval_1.json', 'r') as json_file:
    train_1 = json.load(json_file)

with open('eval_2.json', 'r') as json_file:
    train_2 = json.load(json_file)

with open('eval_3.json', 'r') as json_file:
    train_3 = json.load(json_file)

import pandas as pd

test_df = pd.read_csv("eval_bos.csv")
# with open(output_path, 'r') as json_file:
#     train_1 = json.load(json_file)

In [ ]:
full_data={}

for idx,row in test_df.iterrows():
    data = {
        "original_summary":row['human_summaries'],
        "generated_summaries":[train_1[row['video_ids']],train_2[row['video_ids']],train_3[row['video_ids']]]
                               }
    full_data[row['video_ids']] = data

In [ ]:
import openai

# Initialize API key
openai.api_key = "Provide your key"
# # Inputs
# original_summary = "The discussion is about an interview with Michael Gayed, a portfolio manager and author. When asked about how he defines success, he mentions that it is best defined as an outcome where you don't regret your decisions. He emphasizes that success is not necessarily about money or the size of your company, but rather the absence of regret. When asked about his fund management style, he mentions the importance of diversification in his strategies and having a balanced portfolio to minimize concentration risk. He also emphasizes the importance of communication and having one-on-one conversations in a world of zeros and ones."
# generated_summaries = [
#     "Michael Gayed, a fund manager, defines success as an outcome where one doesn't regret their decisions. He believes that success is more about regret than anything else and that it is defined in different ways for everyone. He also values diversification and having multiple funds with equal asset growth to avoid concentration risk. His goal is to have a smoother cash flow stream and to communicate with investors on a one-on-one basis.",
#     "The discussion revolves around the concept of success and how it is defined differently for everyone. The speaker believes that success is more about not regretting the outcome, even if it is not necessarily about money or the size of a company. He also talks about the importance of diversification in the fund management world and how it can provide a smoother cash flow stream. The speaker mentions his Twitter handle and website for those interested in learning more about him and his funds.",
#     "Michael Gayed, a fund manager, defines success as an outcome where one doesn't regret their decisions. He believes that success is more about regret than anything else and that it is defined in different ways for everyone. He also values diversification and having multiple funds with equal asset growth to avoid concentration risk. His goal is to have a smoother cash flow stream and to communicate with investors on a one-on-one basis.",
#     #"Generated summary 4 text."
# ]

# Ranking function
def rank_summaries_by_index(original_summary, generated_summaries):
    # Create a consolidated prompt
    prompt = f"""
You are an expert in ranking generated summaries based on their alignment, relevance, and coherence to a given reference summary.If its not possible to rank them just check which summary is close to the original. I dont want exaplanation just ranking only.
Original Summary:
{original_summary}

Generated Summaries:
1. {generated_summaries[0]}
2. {generated_summaries[1]}
3. {generated_summaries[2]}

Task:
Rank the generated summaries from 1 (best) to 4 (worst) based on how well they align with the original summary.
Output only the indices of the summaries in ranked order, separated by spaces.
For example: "2 3 1 4" (best to worst).
"""
    # Call GPT
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use GPT-4 for better reasoning
        messages=[
            {"role": "system", "content": "You are a ranking and evaluation assistant."},
            {"role": "user", "content": prompt},
        ],
    )
    return response["choices"][0]["message"]["content"].strip()

# Rank the summaries
# ans = {}
from tqdm.notebook import tqdm
for key in tqdm(full_data):
    if key not in ans:

        generated_summaries = full_data[key]['generated_summaries']
        original_summary = full_data[key][ "original_summary"]


        ranked_indices = rank_summaries_by_index(original_summary, generated_summaries)
        ans[key]= ranked_indices
    # elif key in selected_keys:
    #     generated_summaries = full_data[key]['generated_summaries']
    #     original_summary = full_data[key][ "original_summary"]


    #     ranked_indices = rank_summaries_by_index(original_summary, generated_summaries)
    #     ans[key]= ranked_indices

# # Display the ranked indices
# print("Ranked Indices (Best to Worst):")
# print(ranked_indices)


  0%|          | 0/84 [00:00<?, ?it/s]